In [1]:
from tensorflow.keras.models import load_model
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
model = load_model("10k_2L_ms_ssim_10f(avgpool,k2=0.06,f=5).keras", safe_mode=False) #Enter file path of save model

In [11]:
def prepare_dataframe(input_folder):
    image_dir = input_folder
    file_path =  [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('JPG','PNG','JPEG','jpeg', 'jpg'))]
    df = pd.DataFrame({
        'filename' : file_path,
        'class' : [0] * len(file_path)
    })
    
    return df

In [12]:
df_test=prepare_dataframe("model_trial_result/test_img") #file path of test image folder

In [13]:
df_test

,filename,class
0,model_trial_result/test_img\Parrot.jpg,0
1,model_trial_result/test_img\Test 7.jpg,0


In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_img = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        target_size=(1024, 1024),
        batch_size=64,
        color_mode='grayscale',
        class_mode="input",
        shuffle=False,
        workers=8,
        use_multiprocessing=True 
    )

Found 2 validated image filenames.


In [7]:
predictions = model.predict(test_img)

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step


In [10]:
save_dir = 'model_trial_result/predicted_img'
os.makedirs(save_dir, exist_ok=True)
for i, prediction in enumerate(predictions):
    original_image_path = test_img.filepaths[i]
    image_name = os.path.basename(original_image_path)
    predicted_image = (predictions[i]*255).astype(np.float32)
    trial_no = '20'
    name, ext = os.path.splitext(image_name)
    predicted_image_name = f'pred_{name}_trial-{trial_no}{ext}'
    predicted_image_path = os.path.join(save_dir, predicted_image_name)
    cv2.imwrite(predicted_image_path, predicted_image)